In [1]:
from CyRK import pysolve_ivp, WrapCySolverResult, __version__

print(__version__)

import Cython

0.11.6a0.dev2


In [2]:
%load_ext cython

In [ ]:
%%cython -a -f
# distutils: language = c++
# cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True, initializedcheck=False

import numpy as np
cimport numpy as np
np.import_array()

from libcpp.vector cimport vector

from CyRK cimport cysolve_ivp, CySolveOutput, CySolverResult, DiffeqFuncType, PreEvalFunc


cdef void lotkavolterra_diffeq(double* dy_ptr, double t, double* y_ptr, char* args_ptr, PreEvalFunc pre_eval_func) noexcept nogil:
    # Unpack args
    cdef double* args_dbl_ptr = <double*>args_ptr
    cdef double a = args_dbl_ptr[0]
    cdef double b = args_dbl_ptr[1]
    cdef double c = args_dbl_ptr[2]
    cdef double d = args_dbl_ptr[3]

    if t == 4.38:
        printf("\tDIFFEQ! args ptr = %p. a = %f, b = %f, c = %f, d = %f\n", args_dbl_ptr, a, b, c, d)

    # Unpack y
    cdef double y0, y1
    y0 = y_ptr[0]
    y1 = y_ptr[1]

    dy_ptr[0] = a * y0 - b * y0 * y1
    dy_ptr[1] = -c * y1 + d * y0 * y1

    dy_ptr[2] = a + b + c
    dy_ptr[3] = b + 2.0 * c

cdef DiffeqFuncType diffeq = lotkavolterra_diffeq


cdef CySolveOutput test_1():

    cdef double[2] t_span = [0., 15.]
    cdef double* t_span_ptr = &t_span[0]

    cdef double[2] y0 = [10., 5.]
    cdef double* y0_ptr = &y0[0]

    cdef size_t num_y = 2

    cdef vector[double] args = vector[double]()
    args.resize(4)
    args[0] = 1.5
    args[1] = 1.0
    args[2] = 3.0
    args[3] = 1.0
    cdef double* args_ptr = args.data()

    cdef CySolveOutput result = cysolve_ivp(
        diffeq,
        t_span_ptr,
        y0_ptr,
        num_y,
        1,
        1.0e-2,
        1.0e-3,
        args_ptr,
        sizeof(double) * 4,
        2,
        1_000_000,
        2_000,
        True,
        NULL,
        0,
        NULL,
        NULL,
        NULL,
        10_000,
        0.0,
        100
        )
    return result

from libc.stdlib cimport malloc, free, realloc

cdef CySolveOutput test_2():

    cdef double[2] t_span = [0., 10.]
    cdef double* t_span_ptr = &t_span[0]

    cdef double[2] y0 = [10., 5.]
    cdef double* y0_ptr = &y0[0]

    cdef size_t num_y = 2

    cdef size_t arg_size = sizeof(double)*4
    cdef double* args_ptr = <double*>malloc(arg_size)
    args_ptr[0] = 1.5
    args_ptr[1] = 1.0
    args_ptr[2] = 3.0
    args_ptr[3] = 1.0

    printf("Test 2 Pt 1\n")
    printf("Test 2 Pt 1; args = %p\n", args_ptr)
    cdef CySolveOutput result = cysolve_ivp(
        diffeq,
        t_span_ptr,
        y0_ptr,
        num_y,
        1,
        1.0e-2,
        1.0e-3,
        args_ptr,
        arg_size,
        2,
        1_000_000,
        2_000,
        True,
        NULL,
        0,
        NULL,
        NULL,
        NULL,
        10_000,
        0.0,
        100
        )
    printf("Test 2 Pt 2\n")
    args_ptr = <double*>realloc(args_ptr, sizeof(double)*3000)
    printf("Test 2 Pt 3\n")
    cdef size_t i 
    for i in range(3000):
        args_ptr[i] = -99.0
    printf("Test 2 Pt 4; args = %p\n", args_ptr)
    free(args_ptr)
    args_ptr = NULL
    printf("Test 2 Pt 5\n")
    return result

cdef CySolveOutput res_shptr
cdef CySolverResult* res

from libc.stdio cimport printf

cdef double[4] y_interp
cdef double* y_interp_ptr = &y_interp[0]
cdef size_t i

printf("\nTest 1\n")
for i in range(10):
    printf("\tSubTest Num = %d\n", i)
    res_shptr = test_1()
    res = res_shptr.get()
    printf("Test 1 Success = %d\n", res.success)
    
    y_interp_ptr[0] = 0.0
    y_interp_ptr[1] = 0.0
    y_interp_ptr[2] = 0.0
    y_interp_ptr[3] = 0.0
    
    printf("Test 1; calling\n")
    res.call(4.38, y_interp)
    printf("Test 1; Call Finished. y0 = %e; y1 = %e; y2 = %e; y3 = %e\n", y_interp[0], y_interp[1], y_interp[2], y_interp[3])

printf("\nTest 2\n")
for i in range(10):
    printf("\tSubTest Num = %d\n", i)
    res_shptr = test_2()
    res = res_shptr.get()
    printf("Test 2 Success = %d\n", res.success)
    
    y_interp_ptr[0] = 0.0
    y_interp_ptr[1] = 0.0
    y_interp_ptr[2] = 0.0
    y_interp_ptr[3] = 0.0
    
    printf("Test 2; calling\n")
    res.call(4.38, y_interp)
    printf("Test 2; Call Finished. y0 = %e; y1 = %e; y2 = %e; y3 = %e\n", y_interp[0], y_interp[1], y_interp[2], y_interp[3])

Content of stdout:
_cython_magic_c0c2a34c718801b06ca16ae83c6a67ae4f243f33.cpp
C:\Users\joepr\anaconda3\envs\cyrk11py311\Lib\site-packages\numpy\core\include\numpy\npy_1_7_deprecated_api.h(14) : Warning Msg: Using deprecated NumPy API, disable it with #define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
C:\Users\joepr\anaconda3\envs\cyrk11py311\Lib\site-packages\CyRK\cy\dense.cpp(155): warning C5051: attribute [[unlikely]] requires at least '/std:c++20'; ignored
C:\Users\joepr\anaconda3\envs\cyrk11py311\Lib\site-packages\CyRK\cy\cysolution.cpp(76): warning C5051: attribute [[unlikely]] requires at least '/std:c++20'; ignored
C:\Users\joepr\anaconda3\envs\cyrk11py311\Lib\site-packages\CyRK\cy\cysolution.cpp(122): warning C5051: attribute [[unlikely]] requires at least '/std:c++20'; ignored
C:\Users\joepr\anaconda3\envs\cyrk11py311\Lib\site-packages\CyRK\cy\cysolution.cpp(234): warning C5051: attribute [[unlikely]] requires at least '/std:c++20'; ignored
C:\Users\joepr\anaconda3\envs\cyrk11p